In [3]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

gpt_model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [4]:
from latent import *
model = LatentLM("gpt2", model=gpt_model, tokenizer=tokenizer)

In [5]:
from latent import *
class Adapter(LatentModule):
    def __init__(self):
        super().__init__()
        
        self.lin1 = torch.nn.Linear(768, 768)

    def forward(self, x):
        x = self.lin1(x)
        return x
    
adapter = Adapter()

In [6]:
def color_match(color: str):
    soft_color = model(f"The boat is {color}.", name='soft_color')[-2]
    soft_color = adapter(soft_color)
    x = model(soft_color, "The color of the boat is", name='soft_color')
    
    y = model(f"The boat is {color}. The color of the boat is", name='hard_color')
    
    return x[-1] == y[-1]
color_match("red")

LatentEqualityObjective:
 - lat.Latent[soft_color] <-> ' is[TOK]'
 - lat.Latent[hard_color] <-> ' is[TOK]'

In [7]:
color = "red"
model(f"The boat is {color}. The color of the boat is", name='hard_color').complete()

lat.Latent[12] <-> 'The boat is red. The color of the boat is red[TOK]'

In [9]:
color = "white"
soft_color = model(f"The boat is {color}.", name='soft_color')[-2]
soft_color = adapter(soft_color)
x = model(soft_color, "The color of the boat is", name='soft_color').complete()
x

lat.Latent[14] <-> '[Adapter(soft_color)]The color of the boat is a[TOK]'

In [11]:
colors = ["red", "blue", "orange", "black", "white", "pink"]
adapter.fit(color_match, colors, epochs=100, lr=1e-4)

Loss: 0.0868 Token match: 0.00%


Loss: 0.1277 Token match: 0.00%


Loss: 0.1372 Token match: 0.00%


Loss: 0.1609 Token match: 0.00%


Loss: 0.1541 Token match: 0.00%


...

Loss: 0.0755 Token match: 0.00%


Loss: 0.0746 Token match: 0.00%


Loss: 0.0738 Token match: 0.00%


Loss: 0.0776 Token match: 0.00%


KeyboardInterrupt: 